In [1]:
import pandas as pd
import os
from pathlib import Path
import boto3
import datetime
from datetime import timedelta
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Get the project root directory (go up one level from the notebook directory)
project_root = Path(os.getcwd()).resolve().parent.parent

# Define paths relative to project root
data_planta_dir = os.path.join(project_root, 'data_planta')
planta_file = os.path.join(data_planta_dir, 'Estado Planta 25 JUNIO  2025 .xlsm')
gantt_file = os.path.join(data_planta_dir, 'Gantt-Laser 16 Junio.xlsx')
output_file = os.path.join(data_planta_dir, 'df_planif.xlsx')

# Read the Excel files
planta_df = pd.read_excel(planta_file, skiprows=1)
# gantt_df = pd.read_excel(gantt_file, skiprows=3)

/Users/joserubio/Desktop/proyectos/Kupfer/StreamlitsApps/CNC/dashboard_cnc/.venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [2]:
# Display first few rows and column names
print("\nPlanta DataFrame:")
planta_df


Planta DataFrame:


,DESPPLNF,Dias Atraso (SAP),PV,CLIENTE,Tipo,Of. Destino,DESP. \nPLANIF.,COMPR.\n SAP,Corte,Plegado,Cilindrado,Pernos Stud,Mecanizado,Bisel,Armado,Viga,Externo
0,-141.0,141,55699805,EQUIPOS DE IZAJE PROWINCH CHILE SPA,Complejo,-,04-02-2025,2025-02-04 00:00:00,0,0,0,0,0,0,1,0,1
1,NaN,41,55809850,TECNOLOGIAS INDUSTRIALES SPA,Complejo,Antofagasta,Sin Inf.,2025-05-15 00:00:00,0,0,0,0,0,1,0,0,0
2,NaN,40,55794759,CORP NACIONAL DEL COBRE DE CHILE,Complejo,CD,Sin Inf.,2025-05-16 00:00:00,0,0,0,0,0,0,0,0,6
3,-19.0,28,55816226,JOY GLOBAL CHILE S.A.,Complejo,Antofagasta,06-06-2025,2025-05-28 00:00:00,0,0,0,0,0,0,1,0,0
4,NaN,27,55793184,MAESTRANZA VERDUGO LIMITADA,Complejo,CD,Sin Inf.,2025-05-29 00:00:00,0,0,0,0,0,0,0,186,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,NaN,-,ZO0000001191,(en blanco),Complejo,CD,Sin Inf.,-,2,0,0,0,0,0,0,0,0
180,NaN,-,ZO0000001190,(en blanco),Complejo,CD,Sin Inf.,-,1,0,0,0,0,0,0,0,0
181,NaN,-,ZO0000001193,(en blanco),Complejo,CD,Sin Inf.,-,1,0,0,0,0,0,0,0,0
182,NaN,-,ZO0000001189,(en blanco),Complejo,CD,Sin Inf.,-,0,0,0,0,0,1,0,0,0


In [3]:
# # Display Gantt DataFrame
# print("Gantt DataFrame:")
# gantt_df.head(5)

In [4]:
# Create a function to get unique PV values with their corresponding 'DESP. \nPLANIF.' values
def get_pv_with_planif():
    # Create a dictionary of PV to DESP. \nPLANIF. mappings
    pv_planif_dict = dict(zip(planta_df['PV'], planta_df['DESP. \nPLANIF.']))

    # Get unique PVs
    unique_pvs = planta_df['PV'].unique()

    # Create a dataframe with unique PVs and their PLANIF values
    result = pd.DataFrame({
        'PV': unique_pvs,
        'DESP. PLANIF.': [pv_planif_dict.get(pv) for pv in unique_pvs]
    })

    return result

# Get the PV to PLANIF mapping
pv_planif = get_pv_with_planif()
print(f"Found {len(pv_planif)} unique PV values")
pv_planif

Found 184 unique PV values


,PV,DESP. PLANIF.
0,55699805,04-02-2025
1,55809850,Sin Inf.
2,55794759,Sin Inf.
3,55816226,06-06-2025
4,55793184,Sin Inf.
...,...,...
179,ZO0000001191,Sin Inf.
180,ZO0000001190,Sin Inf.
181,ZO0000001193,Sin Inf.
182,ZO0000001189,Sin Inf.


In [5]:
# Set up AWS session
session = boto3.Session(profile_name='kubeprofile', region_name='us-east-1')

def get_all_items_from_table(table_name):
    dynamo = boto3.resource('dynamodb', region_name='us-east-1')
    table = dynamo.Table(table_name)

    response = table.scan()
    items = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    return items

laser = get_all_items_from_table('sam-stack-irlaa-LaserUploadTable-17V9411WFQMR0')
laser_df = pd.DataFrame(laser)
laser_df = laser_df[laser_df['status'] == 'active']
laser_df = laser_df.rename(columns={'pv': 'PV'})
laser_df

,cliente,cnc,metros,PV,status,createdAt,time,kg,espesor
31,takraf,022770,5,55663774,active,2025-06-24T20:11:36.164Z,4,19,12
32,takraf,022771,7,55663774,active,2025-06-24T20:10:58.284Z,18,72,14
33,takraf,022772,2,55663774,active,2025-06-24T20:09:25.762Z,8,9,16
35,takraf,022774,6,55663774,active,2025-06-24T20:08:39.141Z,24,69,25
36,takraf,022923,7,55663774,active,2025-06-24T20:13:30.661Z,8,7,2
37,takraf,022924,5,55663774,active,2025-06-24T20:04:15.043Z,6,7,3
38,takraf,022925,7,55663774,active,2025-06-24T20:03:38.423Z,8,16,5
39,takraf,022926,7,55663774,active,2025-06-24T20:02:59.124Z,8,39,10
41,takraf,022515,19,55844847,active,2025-06-17T17:00:10.745Z,7,84,6
42,takraf,022527,85,55844847,active,2025-06-10T12:35:35.532Z,20.88,54,2


In [6]:
# Convert both PV columns to string type for consistent comparison
pv_planif['PV'] = pv_planif['PV'].astype(str)
laser_df['PV'] = laser_df['PV'].astype(str)

# Now try the merge again
df_planif = laser_df.merge(pv_planif, on='PV', how='left')

# Drop unnecessary columns and sort
df_planif.drop(['status', 'createdAt'], axis=1, inplace=True)
df_planif.sort_values(by=['DESP. PLANIF.'], ascending=True, inplace=True)

# Save to Excel first
df_planif.to_excel(output_file, index=False)

In [7]:
# Apply color coding based on dates

# Load the workbook
workbook = load_workbook(output_file)
ws = workbook.active

# Define colors
red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')  # Red
green_fill = PatternFill(start_color='FF00FF00', end_color='FF00FF00', fill_type='solid')  # Green
blue_fill = PatternFill(start_color='FF0000FF', end_color='FF0000FF', fill_type='solid')  # Blue

# Get today's date
today = datetime.datetime.now().date()

# Find the column index for 'DESP. PLANIF.'
date_col_idx = None
for idx, col in enumerate(ws[1]):
    if col.value == 'DESP. PLANIF.':
        date_col_idx = idx + 1  # +1 because openpyxl is 1-indexed
        break

if date_col_idx:
    # Start from row 2 (skip header)
    for row in range(2, ws.max_row + 1):
        cell = ws.cell(row=row, column=date_col_idx)
        date_str = cell.value

        if date_str and date_str != 'Sin Inf.':
            try:
                # Parse the date (assuming format is DD-MM-YYYY)
                date_parts = date_str.split('-')
                if len(date_parts) == 3:
                    day, month, year = map(int, date_parts)
                    date_value = datetime.date(year, month, day)

                    # Apply color based on date comparison
                    if date_value < today:
                        # Past date - Red
                        for col in range(1, ws.max_column + 1):
                            ws.cell(row=row, column=col).fill = red_fill
                    elif date_value <= today + timedelta(days=2):
                        # Today or next 2 days - Green
                        for col in range(1, ws.max_column + 1):
                            ws.cell(row=row, column=col).fill = green_fill
                    else:
                        # Future date beyond 2 days - Blue
                        for col in range(1, ws.max_column + 1):
                            ws.cell(row=row, column=col).fill = blue_fill
            except Exception as e:
                print(f"Error processing date {date_str}: {e}")

# Save the workbook with formatting
workbook.save(output_file)
print(f"Excel file with color coding saved as '{output_file}'")

Excel file with color coding saved as '/Users/joserubio/Desktop/proyectos/Kupfer/Casos/jamesKupfer/data_planta/df_planif.xlsx'
